In [84]:
import pyopencl as cl
import pyopencl.cltypes
import numpy as np

In [13]:
exp_min = -14
exp_max = 8
num_axis_points = 500

axis = np.logspace( exp_min, exp_max, num_axis_points, base=10.0, dtype=np.float64)

In [14]:
from numpy.random import Generator, MT19937
seed = 20210515
prng = Generator(MT19937(seed))

In [18]:
def get_all_devices():
    devices = []
    pp = cl.get_platforms()
    for p in pp:
        dd = p.get_devices()
        for d in dd:
            devices.append( d )
    return devices

In [88]:
def run_benchmark( ctx, kernel, x, y, x_out, y_out, num_repetitions=3):
    assert len( x ) > 0
    assert len( x ) == len( y )
    assert len( x ) == len( x_out )
    assert len( x ) == len( y_out )
    times = np.zeros( num_repetitions )
    
    if num_repetitions < 1:
        num_repetitions = 1
        
    with cl.CommandQueue( ctx, properties=cl.command_queue_properties.PROFILING_ENABLE ) as queue:
        x_out_arg = cl.Buffer( ctx, cl.mem_flags.READ_WRITE, size=x_out.nbytes )
        y_out_arg = cl.Buffer( ctx, cl.mem_flags.READ_WRITE, size=y_out.nbytes )
        x_arg = cl.Buffer( ctx, cl.mem_flags.READ_ONLY, size=x.nbytes )
        y_arg = cl.Buffer( ctx, cl.mem_flags.READ_ONLY, size=y.nbytes )
        
        
        for ii in range( num_repetitions ):
            if ii % 10 == 0:
                print( f"start repetition {(ii+1):3d} / {num_repetitions:3d}" )
            cl.enqueue_copy( queue, x_arg, x )        
            ev = cl.enqueue_copy( queue, y_arg, y )
            queue.finish()
    
            kernel.set_args( cl.cltypes.long( len( x ) ), x_arg, y_arg, x_out_arg, y_out_arg )
            ev = cl.enqueue_nd_range_kernel( queue, kernel, x.shape, None )
            queue.finish()
            ev.wait()            
            times[ ii ] = ( ev.profile.end - ev.profile.start ) * 1e-9     
        
        del x_out_arg
        del x_arg
        del y_out_arg
        del y_arg
    return times

In [67]:
devices = get_all_devices()
print( f"number of devices found: {len(devices)}" )

used_device_ids = [ 0, ]
used_devices    = { ii: devices[ ii ] for ii in used_device_ids }
contexts        = { ii: cl.Context(devices=[devices[ii]]) for ii in used_device_ids }

number of devices found: 4


In [64]:
prog_c = { ii: cl.Program( contexts[ ii ], """
#include "cernlib_c/ErrorFunctions.h"

__kernel void eval_cerrf( 
    long int const num_arguments,
    __global double const* __restrict x_vec, 
    __global double const* __restrict y_vec, 
    __global double* __restrict x_out_vec, 
    __global double* __restrict y_out_vec )
{
    long int const ii =  get_global_id( 0 );
    if( ii < num_arguments )
    {
        double x_out, y_out;        
        cerrf( x_vec[ ii ], y_vec[ ii ], &x_out, &y_out );
        x_out_vec[ ii ] = x_out;
        y_out_vec[ ii ] = y_out;
    }
}""") for ii in used_device_ids }

for ii in used_device_ids:
    prog_c[ ii ].build( options = [ "-D_GPUCODE=1", "-DCERRF_NO_SYSTEM_INCLUDES=1" ] )

In [65]:
prog_c_rev1 = { ii: cl.Program( contexts[ ii ], """
#include "cernlib_rev_c/ErrorFunctions.h"

__kernel void eval_cerrf( 
    long int const num_arguments,
    __global double const* __restrict x_vec, 
    __global double const* __restrict y_vec, 
    __global double* __restrict x_out_vec, 
    __global double* __restrict y_out_vec )
{
    long int const ii =  get_global_id( 0 );
    if( ii < num_arguments )
    {
        double x_out, y_out;        
        cerrf_rev( x_vec[ ii ], y_vec[ ii ], &x_out, &y_out );
        x_out_vec[ ii ] = x_out;
        y_out_vec[ ii ] = y_out;
    }
}""") for ii in used_device_ids }

for ii in used_device_ids:
    prog_c_rev1[ ii ].build( options = [ "-D_GPUCODE=1", "-DCERRF_NO_SYSTEM_INCLUDES=1" ] )

In [66]:
prog_c_rev2 = { ii: cl.Program( contexts[ ii ], """
#include "cernlib_rev2_c/ErrorFunctions.h"

__kernel void eval_cerrf( 
    long int const num_arguments,
    __global double const* __restrict x_vec, 
    __global double const* __restrict y_vec, 
    __global double* __restrict x_out_vec, 
    __global double* __restrict y_out_vec )
{
    long int const ii =  get_global_id( 0 );
    if( ii < num_arguments )
    {
        double x_out, y_out;        
        cerrf_rev2( x_vec[ ii ], y_vec[ ii ], &x_out, &y_out );
        x_out_vec[ ii ] = x_out;
        y_out_vec[ ii ] = y_out;
    }
}""") for ii in used_device_ids }

for ii in used_device_ids:
    prog_c_rev2[ ii ].build( options = [ "-D_GPUCODE=1", "-DCERRF_NO_SYSTEM_INCLUDES=1" ] )

In [ ]:
import numpy
from numpy.random import Generator, MT19937

full_random_times = { "c":{}, "rev1":{}, "rev2":{} }
full_random_info  = { "c":{}, "rev1":{}, "rev2":{} }
num_args = np.array( [ 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000 ] ) * 1024
EPS  = float( 2.2e-16 )
prng = Generator(MT19937(20150512))

print( "*** cernlib_c kernel:" )

for ii in used_device_ids:
    for N in num_args:
        x = prng.uniform( -1e3, +1e3 + EPS, N )
        y = prng.uniform( -1e3, +1e3 + EPS, N )
        x_out = np.zeros( N )
        y_out = np.zeros( N )
        
        print( f"*** N = {N:8d}" ) 
        kernel = prog_c[ ii ].eval_cerrf
        full_random_info[ "c" ][ "WORK_GROUP_SIZE" ] = kernel.get_work_group_info( 
            cl.kernel_work_group_info.WORK_GROUP_SIZE, devices[ ii ] )
        full_random_info[ "c" ][ "PRIVATE_MEM_SIZE" ] = kernel.get_work_group_info(
            cl.kernel_work_group_info.PRIVATE_MEM_SIZE, devices[ ii ] )
        full_random_times[ "c" ][ N ] = run_benchmark( 
            contexts[ii], kernel, x, y, x_out, y_out, num_repetitions=50 )

print( "*** cernlib_rev1_c kernel:" )

for ii in used_device_ids:
    for N in num_args:
        x = prng.uniform( -1e3, +1e3 + EPS, N )
        y = prng.uniform( -1e3, +1e3 + EPS, N )
        x_out = np.zeros( N )
        y_out = np.zeros( N )
        
        print( f"*** N = {N:8d}" ) 
        kernel = prog_c_rev1[ ii ].eval_cerrf
        full_random_info[ "rev1" ][ "WORK_GROUP_SIZE" ] = kernel.get_work_group_info( 
            cl.kernel_work_group_info.WORK_GROUP_SIZE, devices[ ii ] )
        full_random_info[ "rev1" ][ "PRIVATE_MEM_SIZE" ] = kernel.get_work_group_info(
            cl.kernel_work_group_info.PRIVATE_MEM_SIZE, devices[ ii ] )
        full_random_times[ "rev1" ][ N ] = run_benchmark( 
            contexts[ii], kernel, x, y, x_out, y_out, num_repetitions=50 )
        
print( "*** cernlib_rev2_c kernel:" )

for ii in used_device_ids:
    for N in num_args:
        x = prng.uniform( -1e3, +1e3 + EPS, N )
        y = prng.uniform( -1e3, +1e3 + EPS, N )
        x_out = np.zeros( N )
        y_out = np.zeros( N )
        
        print( f"*** N = {N:8d}" ) 
        kernel = prog_c_rev2[ ii ].eval_cerrf
        full_random_info[ "rev2" ][ "WORK_GROUP_SIZE" ] = kernel.get_work_group_info( 
            cl.kernel_work_group_info.WORK_GROUP_SIZE, devices[ ii ] )
        full_random_info[ "rev2" ][ "PRIVATE_MEM_SIZE" ] = kernel.get_work_group_info(
            cl.kernel_work_group_info.PRIVATE_MEM_SIZE, devices[ ii ] )
        full_random_times[ "rev2" ][ N ] = run_benchmark( 
            contexts[ii], kernel, x, y, x_out, y_out, num_repetitions=50 )
